# Arithmetic conversion: *folding* and *casting*

Apart from graph rewriting, fake-to-true conversions require reordering arithmetic operations, folding together some operands, and casting the results to integer or other low-precision data formats, while at the same time preserving functional equivalence (or at least minimising the discrepancy between the original version and the converted one).
We refer to this part of the conversion as *arithmetic conversion*.


### *Folding*

We call *folding* each sequence of applications of basic arithmetical properties or identity decompositions used to transform a given sequence of operations into an equivalent one.
Commonly-used arithmetical properties are:

* the *commutative* property: $a + b = b + a$;
* the *associative* property: $(a + b) + c = a + (b + c)$;
* the *distributive* property and its inverse: $(a + b) * c = (a * c) + (b * c)$ and $(a * c) + (b * c) = (a + b) * c$.

Given a non-zero number $z \in \mathbb{R}$, the identity decompositions are the following:

* *sum-and-subtract*: $x = x - z + z$;
* *divide-and-multiply*: $ x = (x / z) * z$.

For example, consider the operation of normalising a scalar product.
Let $n > 0$ be an integer, $\mathbb{R}^{n}$ be the $n$-dimensional Euclidean space, and $\mathbf{x}, \mathbf{w} \in \mathbb{R}^{n}$ be vectors.
Let also $\mu \in \mathbb{R},\, \sigma \in \mathbb{R}^{+}$, and consider the following operation:

\begin{equation*}
    \frac{\langle \mathbf{x}, \mathbf{w} \rangle - \mu}{\sigma} \,.
\end{equation*}

Folding rules (in this specific case, the distributive property) allow us to rewrite this operation as

\begin{equation*}
\begin{split}
    \frac{\langle \mathbf{x}, \mathbf{w} \rangle - \mu}{\sigma}
    &= \left( \langle \mathbf{x}, \mathbf{w} \rangle - \mu \right) \frac{1}{\sigma} \\
    &= \langle \mathbf{x}, \mathbf{w} \rangle \frac{1}{\sigma} - \frac{\mu}{\sigma} \\
    &= \langle \mathbf{x}, \mathbf{w} \rangle \tilde{\sigma} + \tilde{\mu} \,.
\end{split}
\end{equation*}

The critical point here is that the operands $\tilde{\sigma} = 1/\sigma \in \mathbb{R}^{+}$ and $\tilde{\mu} = -\mu/\sigma$ can be **pre-computed** starting from the operands $\mu$ and $\sigma$.
The original version of the operation involves computing the scalar product $\langle \mathbf{x}, \mathbf{w} \rangle = \sum_{i=1}^{n} x_{i} w_{i}$ ($n$ multiplications and $n-1$ sums), one subtraction, and one division.
By contrast, the folded version still requires computing the scalar product, but then uses a multiplication and an addition.

In this case, we are still performing the same number of operations, although with different operands.
In other cases, though, the benefits of folding can become more apparent.
For example, instead of supposing $\mathbf{x}, \mathbf{w} \in \mathbb{R}^{n}$, we can imagine that the components of these vectors take values in specific spaces of the form

\begin{equation*}
    \mathbb{Z}_{\epsilon} := \{ \epsilon i \,|\, i \in \mathbb{Z} \} \,,
\end{equation*}

where $\epsilon \in \mathbb{R}^{+}$ is a positive constant.
In particular, given constants $\epsilon_{\mathbf{x}}, \epsilon_{\mathbf{w}} > 0$ we assume that $\mathbf{x} \in \mathbb{Z}_{\epsilon_{\mathbf{x}}}^{n}$ and $\mathbf{w} \in \mathbb{Z}_{\epsilon_{\mathbf{w}}}^{n}$.
Due to the definition of the spaces $\mathbb{Z}_{\epsilon}$, we can make explicit the fact that each $\mathbf{x} \in \mathbb{Z}_{\epsilon_{\mathbf{x}}}^{n}$ can be rewritten as

\begin{equation*}
    \mathbf{x} = \epsilon_{\mathbf{x}} \hat{\mathbf{x}} \,,
\end{equation*}

where $\hat{\mathbf{x}} \in \mathbb{Z}^{n}$ if a vector with integer components $\hat{x}_{i} \in \mathbb{Z}, \, i = 1, \dots, n$.
An analogous representation is available for $\mathbf{w} \in \mathbb{Z}_{\epsilon_{\mathbf{w}}}^{n}$: $\mathbf{w} = \epsilon_{\mathbf{w}} \hat{\mathbf{w}}$.

Given parameters $\gamma, \beta \in \mathbb{R}$ and a positive parameter $\epsilon_{\mathbf{s}} > 0$, a conversion that is often required in fake-to-true network transformations is the following:

\begin{equation*}
\begin{split}
    \frac{\left( \frac{\langle \mathbf{x}, \mathbf{w} \rangle - \mu}{\sigma} \right) \gamma + \beta}{\epsilon_{\mathbf{s}}}
    &= \frac{\left( \frac{\langle \epsilon_{\mathbf{x}} \hat{\mathbf{x}}, \epsilon_{\mathbf{w}} \hat{\mathbf{w}} \rangle - \mu}{\sigma} \right) \gamma + \beta}{\epsilon_{\mathbf{s}}} \\
    &= \left( \left( \frac{ \sum_{i=1}^{n} \epsilon_{\mathbf{x}} \hat{x}_{i} \epsilon_{\mathbf{w}} \hat{w}_{i} - \mu}{\sigma} \right) \gamma + \beta \right) \frac{1}{\epsilon_{\mathbf{s}}} \\
    &= \left( \left( \frac{ \epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \sum_{i=1}^{n} \hat{x}_{i} \hat{w}_{i} - \mu}{\sigma} \right) \gamma + \beta \right) \frac{1}{\epsilon_{\mathbf{s}}} \\
    &= \left( \left( \frac{ \epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \langle \hat{\mathbf{x}}, \hat{\mathbf{w}} \rangle - \mu}{\sigma} \right) \gamma + \beta \right) \frac{1}{\epsilon_{\mathbf{s}}} \\
    &= \left( \langle \hat{\mathbf{x}}, \hat{\mathbf{w}} \rangle \left( \frac{\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma}{\sigma} \right) + \left( \frac{\sigma \beta - \mu \gamma}{\sigma} \right) \right) \frac{1}{\epsilon_{\mathbf{s}}} \,.
\end{split}
\end{equation*}

At this point, the folding process can evolve in two directions: the *add-then-multiply*, and the *multiply-then-add*.
The *add-then-multiply* folding strategy divides the first factor by $(\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma)/\sigma$ and multiplies the second by the same value, obtaining

\begin{equation*}
\begin{split}
    \frac{\left( \frac{\langle \mathbf{x}, \mathbf{w} \rangle - \mu}{\sigma} \right) \gamma + \beta}{\epsilon_{\mathbf{s}}}
    &= \left( \langle \hat{\mathbf{x}}, \hat{\mathbf{w}} \rangle + \left( \frac{\sigma \beta - \mu \gamma}{\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma} \right) \right) \frac{\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \sigma}{\epsilon_{\mathbf{s}} \sigma} \\
    &= \left( \langle \hat{\mathbf{x}}, \hat{\mathbf{w}} \rangle + \beta' \right) \gamma' \,,
\end{split}
\end{equation*}

where $\beta' = (\sigma \beta - \mu \gamma)/(\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma)$ and $\sigma' = (\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma)/(\epsilon_{\mathbf{s}} \sigma)$.
The *multiply-then-add* folding strategy directly multiplies the first factor by $1/\epsilon_{\mathbf{s}}$, yielding

\begin{equation*}
\begin{split}
    \frac{\left( \frac{\langle \mathbf{x}, \mathbf{w} \rangle - \mu}{\sigma} \right) \gamma + \beta}{\epsilon_{\mathbf{s}}}
    &= \langle \hat{\mathbf{x}}, \hat{\mathbf{w}} \rangle \left( \frac{\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma}{\epsilon_{\mathbf{s}} \sigma} \right) + \left( \frac{\sigma \beta - \mu \gamma}{\epsilon_{\mathbf{x}} \sigma} \right) \\
    &= \langle \hat{\mathbf{x}}, \hat{\mathbf{w}} \rangle \gamma'' + \beta'' \,,
\end{split}
\end{equation*}

where $\beta'' = (\sigma \beta - \mu \gamma)/(\epsilon_{\mathbf{s}} \sigma)$ and $\sigma'' = \sigma' = (\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma)/(\epsilon_{\mathbf{s}} \sigma)$.

Before folding, the original expression involves $n$ products and $n-1$ sums between *decimal* numbers (assuming that $\epsilon_{\mathbf{x}}$ and $\epsilon_{\mathbf{w}}$ are decimal numbers), a subtraction, a division, a multiplication, an addition, and another division.
After folding, the expression involves $n$ products and $n-1$ sums between *integer* numbers, a multiplication, and an addition.
By folding, we have derived a more compact version of the original expression, saving three operations.
Nevertheless, the main benefit from the perspective of digital arithmetic is that performing multiplications and additions between integers can be much more efficient than performing it between decimal numbers (fixed-point and especially floating-point).

**Warning**: the assumption of folding is that transformations based on the commutative, associative, and distributive properties as well as on identity decompositions yield *exactly* equivalent operations.
In digital arithmetic, this is usually not the case.
For example, consider the possible ways to compute $\sigma'' = \sigma' = (\epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma)/(\epsilon_{\mathbf{s}} \sigma) = \epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \gamma (1/\epsilon_{\mathbf{s}}) (1/\sigma)$.
Assuming that all the numbers in the expression are represented as floating-points, it might for example happen that

\begin{equation*}
    \left( \left( \epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \right) \gamma \right) \left( \frac{1}{\epsilon_{\mathbf{s}}} \frac{1}{\sigma} \right)
    \neq
    \left( \left( \left( \epsilon_{\mathbf{x}} \epsilon_{\mathbf{w}} \right) \gamma \right) \frac{1}{\epsilon_{\mathbf{s}}} \right) \left( \frac{1}{\sigma} \right) \,.
\end{equation*}


### *Casting*

Casting is the process by which digital data formats are converted into other data formats.
An example of casting is transforming decimal numbers represented using a floating-point data format into decimal number represented using a fixed-point data format.
It is usually the case that casting is a non-invertible transformation.


In [2]:
import torch


# operation parameters
n     = 128

eps_x = 2.0
x_hat = torch.randint(low=-1, high=1+1, size=(n,))
x     = eps_x * x_hat

eps_w = 1.0
w_hat = torch.randint(low=-1, high=1+1, size=(n,))
w     = eps_w * w_hat

mi    = -3.0
sigma = 0.5
gamma = 3.0
beta  = 1.5

eps_s = 3.5


def original(x, w, mi, sigma, gamma, beta, eps_s):
    return (((torch.dot(x, w) - mi) / sigma) * gamma + beta) / eps_s


def fold_dotaddandmultiply(eps_x, eps_w, mi, sigma, gamma, beta, eps_s):
    
    gammaprime  = eps_x * eps_w * gamma / (eps_s * sigma)
    betaprime   = (sigma * beta - mi * gamma) / (eps_x * eps_w * gamma)

    return betaprime, gammaprime


def dotaddandmultiply(x_hat, w_hat, betaprime, gammaprime):
    return (torch.dot(x_hat, w_hat) + betaprime) * gammaprime
    

def fold_dotmultiplyandadd(eps_x, eps_w, mi, sigma, gamma, beta, eps_s):

    gammasecond = eps_x * eps_w * gamma / (eps_s * sigma)
    betasecond  = (sigma * beta - mi * gamma) / (eps_s * sigma)
    
    return gammasecond, betasecond


def dotmultiplyandadd(x_hat, w_hat, gammasecond, betasecond):
    return torch.dot(x_hat, w_hat) * gammasecond + betasecond



o   = original(x, w, mi, sigma, gamma, beta, eps_s)
fam = dotaddandmultiply(x_hat, w_hat, *fold_dotaddandmultiply(eps_x, eps_w, mi, sigma, gamma, beta, eps_s))
fma = dotmultiplyandadd(x_hat, w_hat, *fold_dotmultiplyandadd(eps_x, eps_w, mi, sigma, gamma, beta, eps_s))

print("Original value:                {:10.6f}".format(o))
print("Folded add-and-multiply value: {:10.6f}".format(fam))
print("Folded multiply-and-add value: {:10.6f}".format(fma))


Original value:                  5.571429
Folded add-and-multiply value:   5.571429
Folded multiply-and-add value:   5.571429


## Bonus section: investigating sources of numerical discrepancy

2-D arrays: index in batch $b$, feature $i$
input $x[b]$, filter $w[i]$ ($i$-th row of weight matrix in linear layer)

2-D + d arrays: index in batch $b$, feature $i$, position $(p_{0}, \dots, p_{d-1})$

coords = index[2:]

padding = tuple(itertools.chain.from_iterable([(i, i) for i in mod.padding]))
xpadded = nn.functional(x, padding)

a_{i} = p_{i} * s_{i} --- b_{i} = p_{i} * s_{i} + k_{i}
patch = x[b, :, a_{0}:b_{0}, ..., a_{d-1}:b_{d-1}]

mu[i], sigma[i], weight[i], bias[i], eps


In [ ]:
import itertools

from typing import Tuple


def extract_fov_convnd(x:            torch.Tensor,
                       convmod:      torch.nn.modules.conv._ConvNd,
                       idx_in_batch: int,
                       out_position: Tuple[int]):
    """Compute the patch (*field-of-view*, FOV) onto which the convolutional filter should be applied."""

    padding  = tuple(itertools.chain.from_iterable([(p, p) for p in convmod.padding]))
    x_padded = torch.nn.functional.pad(x, padding)
    
    kernel   = convmod.kernel_size
    stride   = convmod.stride
    dilation = 1
    spatial_slices = list(map(lambda t: slice(t[0] * t[1], t[0] * t[1] + t[2], dilation), zip(out_position, stride, kernel)))
    
    fov = x_padded[(slice(idx_in_batch, idx_in_batch + 1, 1), slice(None, None, 1)) + tuple(slice_ for slice_ in spatial_slices)]

    return fov


In [ ]:
def show_steps_convnd_bn_act_fakequantized(eps_x, x, convmod, bnmod, actmod, diff_coords):
    
    idx_in_batch = diff_coords[0]
    out_channel  = diff_coords[1]
    out_position = tuple(diff_coords[2:])
    
    # retrieve operands
    fov    = extract_fov_convnd(x, convmod, idx_in_batch, out_position)

    eps_w  = convmod.eps
    weight = convmod.weight.data[out_channel]
    
    mi    = bnmod.running_mean[out_channel]
    sigma = torch.sqrt(bnmod.running_var[out_channel] + bnmod.eps)
    gamma = bnmod.weight[out_channel]
    beta  = bnmod.bias[out_channel]
    
    eps_s = actmod.eps
    lut   = torch.vstack((torch.hstack((torch.Tensor([-np.inf]), actmod.thresholds)), actmod.quant_levels))
    
#     # adjust for gamma's sign
#     flip    = torch.sign(gamma)
#     weight *= flip
#     gamma  *= flip
    
    # fake-quantized ops
    temp1 = torch.sum(fov * weight)
    temp2 = temp1 * (gamma / sigma)
    temp3 = temp2 + (beta - (gamma / sigma) * mi)
    
    cmp = (temp3 >= lut[0, :]).float()
    idx = (lut.shape[-1] - 1) if cmp[-1] else torch.argmax(cmp[:-1] - cmp[1:])
    temp4 = lut[1, idx]

    temp5 = temp4 * eps_s
    
    print(temp1, "({})".format(temp1 / (eps_x * eps_w)), temp2, temp3, lut, temp4, temp5, "({})".format(temp5 / eps_s))
    
    
def show_steps_convnd_lut_truequantized(x, convmod, lutmod, diff_coords):

    idx_in_batch = diff_coords[0]
    out_channel  = diff_coords[1]
    out_position = tuple(diff_coords[2:])
    
    # retrieve operands
    fov    = extract_fov_convnd(x, convmod, idx_in_batch, out_position)
    weight = convmod.weight.data[out_channel]

    lut   = torch.vstack((torch.hstack((torch.Tensor([-np.inf]), lutmod.tau[:, out_channel].squeeze())), lutmod.quant_levels))
    
    # true-quantized ops
    temp1 = torch.sum(fov * weight)
    cmp   = (temp1 >= lut[0, :]).float()
    idx   = (lut.shape[-1] - 1) if cmp[-1] else torch.argmax(cmp[:-1] - cmp[1:])
    temp2 = lut[1, idx]
    
    print(temp1, lut[0], temp2)
